### Ezt az egészet átraktam Colabba, mert ott Pro előfizetéssel volt elég erőforrásom, hogy futtassam.

bitsandbytes-szal sikerült ott kvantálni

https://colab.research.google.com/drive/1AWF-efJb7woy2Jp1g3QRb0VmER19pQz_?usp=sharing

In [ ]:
%pip install transformers --quiet
%pip install auto-gptq --quiet
%pip install optimum --quiet
%pip install accelerate --quiet

In [ ]:
from transformers import pipeline, AutoTokenizer,  AutoModelForCausalLM
import torch
from transformers import AutoModelForCausalLM
from optimum.gptq import GPTQQuantizer

PULI GPTrio, online tesztkörnyezetben egész jó eredményeket adott
![](docs/pics/puli_gptrio_test.png)
![](docs/pics/puli_gptrio_test2.png)

In [ ]:
model_id = "NYTK/PULI-GPTrio"

tokenizer = AutoTokenizer.from_pretrained(model_id)

Kvantálás GTPQ config alapján

https://huggingface.co/docs/optimum/llm_quantization/usage_guides/quantization

Sok időt és erőforrást vesz ez is igénybe, többször kékhalált kapott a laptopom vagy csak teljesen kifagyott az egész semmire se reagálva.

![](./docs/pics/kékhalál.jpg)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)
quantizer = GPTQQuantizer(bits=4, dataset="c4", block_name_to_quantize = "model.decoder.layers", model_seqlen = 2048)
quantized_model = quantizer.quantize_model(model, tokenizer)

save_folder = "./quantized_models/PULI-GPTrio"
quantizer.save(model,save_folder)

prompt = "Elmesélek egy történetet a nyelvtechnológiáról."
generator = pipeline(task="text-generation", model=quantized_model, tokenizer=tokenizer)

print(generator(prompt)[0]["generated_text"])

Kvantálás PyTorch-csal


![](./docs/pics/torch_q_fail1.png)

Ennyi memóriának kéne lennie (16 gigából nem jön ki?????)

In [ ]:
import torch.quantization as tq

model = AutoModelForCausalLM.from_pretrained(model_id)

model.eval()

quantized_model = tq.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

torch.save(quantized_model.state_dict(), './quantized_models/torch_q/quantized_model.pt')

bitsandbytes-sal

Ram:
![](./docs/pics/model8bit_ram.png)

In [ ]:
%pip install -i https://test.pypi.org/simple/ bitsandbytes
%pip install accelerate

In [ ]:
model_8bit = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True)

Kvantálás nélkül

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id)
prompt = "Elmesélek egy történetet a nyelvtechnológiáról."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
)

gen_text = tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)

Nem tudom futtatni, Google Colabon és lokálisan is Out of memory errort kapok
![](docs/pics/colab_run_error.png)

huBERT fine tuned modell

In [ ]:
from transformers import pipeline
qa_pipeline = pipeline(
    "question-answering",
    model="mcsabai/huBert-fine-tuned-hungarian-squadv2",
    tokenizer="mcsabai/huBert-fine-tuned-hungarian-squadv2",
    top_k = 1,
    handle_impossible_answer = True
)

prompt = {
    'context': "A Python egy általános célú, nagyon magas szintű programozási nyelv, melyet Guido van Rossum holland programozó kezdett el fejleszteni 1989 végén, majd hozott nyilvánosságra 1991-ben.[11] A nyelv tervezési filozófiája az olvashatóságot és a programozói munka megkönnyítését helyezi előtérbe a futási sebességgel szemben.[36][37][38] Például a behúzások szintaktikailag is fontosak. A Python többek között a funkcionális, az objektumorientált, az aspektusorientált az imperatív és a procedurális programozási paradigmákat támogatja. Dinamikus típusokat és automatikus memóriakezelést használ, ilyen szempontból hasonlít a Scheme, Perl és Ruby nyelvekhez, emellett szigorú típusrendszerrel rendelkezik. Erőssége a gazdag szabványos programkönyvtár.[40][41] A Python úgynevezett interpreteres nyelv, ami azt jelenti, hogy nincs különválasztva a forrás- és tárgykód, a megírt program máris futtatható, ha rendelkezünk a Python értelmezővel. A Python értelmezőt számos géptípusra és operációs rendszerre elkészítették, továbbá számtalan kiegészítő könyvtár készült hozzá, így rendkívül széles körben használhatóvá vált. Az egyik legnépszerűbb programozási nyelv.[42][43][44][45] Nyitott, közösségalapú fejlesztési modellt mutat fel, amit a közhasznú Python Software Foundation felügyel, ami a nyelv definícióját a CPython referenciaimplementációval gondozza. ",
    'question': "Ki találta ki a Pythont?"
}

predictions = qa_pipeline(prompt)

print(f'Kérdés: {prompt["question"]}')
print(f'Válasz: {predictions["answer"]}')
print(f'Konfidencia: {predictions["score"]}')
print(predictions)
